In [ ]:
from tkinter.filedialog import askopenfilename
import pandas as pd
import tkinter
import os
import numpy as np

tk_window = tkinter.Tk()
cwd = os.getcwd()

tr_data = pd.read_csv(askopenfilename(title='Choose your training data'))

tr_ans = pd.read_csv(askopenfilename(title='Choose your training answer'))

tr_ans = tr_ans.iloc[:, 0]

ts_data = pd.read_csv(askopenfilename(title='Choose your test data'))

tk_window.destroy()


In [ ]:


# 문제 발생 시 다음 코드로!!
#string_columns = tr_data.dtypes
#string_boolidx = string_columns == np.object
#tr_data = tr_data.drop(tr_data.columns[string_boolidx], axis=1)
#ts_data = ts_data.drop(ts_data.columns[string_boolidx], axis=1)

# one hot encoding 할 column 선택
one_hot_encode_cols = tr_data.dtypes[tr_data.dtypes == np.object]  # filtering by string categoricals
one_hot_encode_cols = one_hot_encode_cols.index.tolist()  # list of categorical fields

# one hot encoding
for col in one_hot_encode_cols:
    tr_data[col] = pd.Categorical(tr_data[col])
tr_data = pd.get_dummies(tr_data, columns=one_hot_encode_cols)
for col in one_hot_encode_cols:
    ts_data[col] = pd.Categorical(ts_data[col])
ts_data = pd.get_dummies(ts_data, columns=one_hot_encode_cols)

# one hot encoding 된 column들의 교집합
tr_data_columns = set(tr_data.columns)
ts_data_columns = set(ts_data.columns)

columns = tr_data_columns & ts_data_columns # 교집합
columns = list(columns)

tr_data = tr_data[columns]
ts_data = ts_data[columns]

# 모델 선택
from sklearn.ensemble import GradientBoostingRegressor
# 1000,0.1,2 random forest보다 좋음
# 800, 0.045, 4 원래 했던거!
# 900, 0.0475, 4
model = GradientBoostingRegressor(n_estimators = 900, learning_rate =0.0475, 
                                 max_depth = 4, random_state=0)

model = model.fit(tr_data, tr_ans)

y_pred = model.predict(ts_data)


pred_df = pd.DataFrame(y_pred)

pred_df.to_csv("output.csv", mode='w')

# End of Demo

In [ ]:
from sklearn.metrics import mean_squared_error


def rmse(ytrue, ypredicted):
    return np.sqrt(mean_squared_error(ytrue, ypredicted))

tk_window = tkinter.Tk()
cwd = os.getcwd()
ts_ans = pd.read_csv(askopenfilename(title='Choose your test answer'))
ts_ans = ts_ans.iloc[:, 0]
tk_window.destroy()

print(rmse(ts_ans, y_pred))